In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
df = pd.read_csv('Toronto2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3L,North York,Downsview West,43.739015,-79.506944
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
3,M8V,Etobicoke,New Toronto,43.605647,-79.501321
4,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [3]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)
map_toronto

In [5]:
CLIENT_ID = 'O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX' # your Foursquare ID
CLIENT_SECRET = 'JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX
CLIENT_SECRET:JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX


In [20]:
n_lat = df.loc[2, 'Latitude']
n_lon = df.loc[2, 'Longitude']
n_name = df.loc[2, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(n_name, n_lat, n_lon))

Latitude and longitude values of Dovercourt Village are 43.66900510000001, -79.4422593.


In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_lat, 
    n_lon, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX&client_secret=JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX&v=20180605&ll=43.66900510000001,-79.4422593&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5b4bdf9ba3e50039aaec51'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Davenport',
  'headerFullLocation': 'Davenport, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.67350510450001,
    'lng': -79.43604977526607},
   'sw': {'lat': 43.664505095500004, 'lng': -79.44846882473394}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5753753b498eeb535c53aed5',
       'name': 'The Greater Good Bar',
       'location': {'address': '229 Geary St',
        'crossStreet': 'at Dufferin St',
        'lat': 43.669409,
        'lng': -79.439267,
        'labeledLatLngs': [{'label': 'disp

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Greater Good Bar,Bar,43.669409,-79.439267
1,Parallel,Middle Eastern Restaurant,43.669516,-79.438728
2,Happy Bakery & Pastries,Bakery,43.667050,-79.441791
3,Planet Fitness Toronto Galleria,Gym / Fitness Center,43.667588,-79.442574
4,FreshCo,Supermarket,43.667918,-79.440754


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


### 2. Explore Neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
df_venues = getNearbyVenues(names=df['Neighborhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude'])

Downsview West
Little Portugal, Trinity
Dovercourt Village
New Toronto
Alderwood, Long Branch
Northwood Park, York University
Thorncliffe Park
Design Exchange, Toronto Dominion Centre
Church and Wellesley
Willowdale West
Highland Creek, Rouge Hill, Port Union
Maryvale, Wexford
Bathurst Manor, Wilson Heights
The Annex, Yorkville
Rosedale
Humber Summit
Hillcrest Village
Newtonbrook, Willowdale
Woburn
Woodbine Heights
Weston
High Park
Moore Park
Victoria Village
Emery, Humberlea
Humber Bay, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Berczy Park
Scarborough Village
Deer Park, Rathnelly, South Hill
Flemingdon Park
Chinatown, Grange Park, Kensington Market
Forest Hill North
Islington Avenue
CFB Toronto
Downsview
Parkdale, Roncesvalles
Ionview, Kennedy Park
Kingsview Village
Runnymede, Swansea
Cabbagetown, St. James Town
Upper Rouge
India Bazaar
Leaside
Birch Cliff
St. James Town
Islington, Princess Gardens, West Deane Park
University of Toronto
Beaumond He

In [27]:
print(df_venues.shape)
df_venues.head(10)

(1577, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downsview West,43.739015,-79.506944,TD Canada Trust,43.740369,-79.512375,Bank
1,Downsview West,43.739015,-79.506944,Giltspur Park,43.735724,-79.507821,Park
2,Downsview West,43.739015,-79.506944,Win Farm Supermarket,43.739193,-79.512053,Grocery Store
3,Downsview West,43.739015,-79.506944,Price Chopper,43.739908,-79.512261,Grocery Store
4,Downsview West,43.739015,-79.506944,jane sheppard mall,43.740104,-79.512552,Shopping Mall
5,"Little Portugal, Trinity",43.647927,-79.419750,Bellwoods Brewery,43.647097,-79.419955,Brewery
6,"Little Portugal, Trinity",43.647927,-79.419750,Pizzeria Libretto,43.648979,-79.420604,Pizza Place
7,"Little Portugal, Trinity",43.647927,-79.419750,OddSeoul,43.646192,-79.419601,Korean Restaurant
8,"Little Portugal, Trinity",43.647927,-79.419750,Foxley Bistro,43.648643,-79.420495,Asian Restaurant
9,"Little Portugal, Trinity",43.647927,-79.419750,Bang Bang Ice Cream & Bakery,43.646246,-79.419553,Ice Cream Shop


In [28]:
df_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North, Milliken",2,2,2,2,2,2
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
Bedford Park,22,22,22,22,22,22
Berczy Park,57,57,57,57,57,57
Birch Cliff,5,5,5,5,5,5


In [29]:
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 254 uniques categories.


### 3. Analyze Each Neighborhood

In [30]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downsview West,0,0,0,0,0,0,0,0,0,

In [31]:
df_onehot.shape

(1577, 254)

In [32]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,

In [33]:
df_grouped.shape

(75, 254)

In [46]:
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0        Skating Rink   0.2
1              Lounge   0.2
2  Chinese Restaurant   0.2
3      Sandwich Place   0.2
4      Breakfast Spot   0.2


----Agincourt North, Milliken----
               venue  freq
0         Playground   0.5
1               Park   0.5
2             Museum   0.0
3             Market   0.0
4  Martial Arts Dojo   0.0


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1            Pool  0.11
2    Skating Rink  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bathurst Manor, Wilson Heights----
                  venue  freq
0           Coffee Shop  0.09
1             Pet Store  0.05
2         Shopping Mall  0.05
3           Bridal Shop  0.05
4  Fast Food Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Beaumond Hei

                  venue  freq
0        Hardware Store  0.07
1         Grocery Store  0.07
2           Flower Shop  0.07
3         Burrito Place  0.07
4  Fast Food Restaurant  0.07


----Moore Park----
           venue  freq
0     Playground  0.25
1            Gym  0.25
2     Restaurant  0.25
3           Park  0.25
4  Metro Station  0.00


----Morningside, West Hill----
                 venue  freq
0          Pizza Place  0.14
1    Electronics Store  0.14
2         Intersection  0.14
3  Rental Car Location  0.14
4   Mexican Restaurant  0.14


----New Toronto----
                  venue  freq
0              Pharmacy  0.07
1            Restaurant  0.07
2             Pet Store  0.07
3  Fast Food Restaurant  0.07
4           Pizza Place  0.07


----Newtonbrook, Willowdale----
               venue  freq
0               Park   1.0
1        Yoga Studio   0.0
2             Museum   0.0
3             Market   0.0
4  Martial Arts Dojo   0.0


----Northwood Park, York University----
              

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Chinese Restaurant,Breakfast Spot,Skating Rink,Sandwich Place,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
1,"Agincourt North, Milliken",Playground,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
2,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Service,Farmers Market
3,"Bathurst Manor, Wilson Heights",Coffee Shop,Pet Store,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Fast Food Restaurant,Diner,Restaurant,Deli / Bodega,Supermarket
4,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


### 4. Cluster Neighborhoods

In [108]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [93]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.fillna(0, inplace=True)

df_merged['Cluster Labels'] = [int(i) for i in df_merged['Cluster Labels']]

df_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3L,North York,Downsview West,43.739015,-79.506944,0,Grocery Store,Bank,Park,Shopping Mall,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Coffee Shop,Asian Restaurant,Men's Store,Restaurant,Café,Pizza Place,Cocktail Bar,New American Restaurant,Bakery
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,0,Pharmacy,Bakery,Supermarket,Coffee Shop,Bank,Brewery,Middle Eastern Restaurant,Café,Music Venue,Pizza Place
3,M8V,Etobicoke,New Toronto,43.605647,-79.501321,0,Pharmacy,Bakery,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pet Store,Pizza Place,Coffee Shop,Restaurant,Café
4,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0,Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Service,Farmers Market
5,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,4,Falafel Restaurant,Coffee Shop,Caribbean Restaurant,Massage Studio,Bar,Metro Station,Women's Store,Dumpling Restaurant,Doner Restaurant,Donut Shop
6,M4H,East York,Thorncliffe Park,43.705369,-79.349372,0,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Gym,Housing Development,Park,Pizza Place,Coffee Shop,Sandwich Place
7,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Bakery,Gym,Bar,American Restaurant,Italian Restaurant
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Bubble Tea Shop,Hotel,Café,Men's Store,Pub
9,M2R,North York,Willowdale West,43.782736,-79.442259,0,Pizza Place,Coffee Shop,Discount Store,Butcher,Pharmacy,Concert Hall,Construction & Landscaping,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Clusters

In [95]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Grocery Store,Bank,Park,Shopping Mall,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Men's Store,Restaurant,Café,Pizza Place,Cocktail Bar,New American Restaurant,Bakery
2,West Toronto,0,Pharmacy,Bakery,Supermarket,Coffee Shop,Bank,Brewery,Middle Eastern Restaurant,Café,Music Venue,Pizza Place
3,Etobicoke,0,Pharmacy,Bakery,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pet Store,Pizza Place,Coffee Shop,Restaurant,Café
4,Etobicoke,0,Pizza Place,Pharmacy,Skating Rink,Coffee Shop,Pool,Pub,Sandwich Place,Gym,Airport Service,Farmers Market
6,East York,0,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Gym,Housing Development,Park,Pizza Place,Coffee Shop,Sandwich Place
7,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Bakery,Gym,Bar,American Restaurant,Italian Restaurant
8,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Bubble Tea Shop,Hotel,Café,Men's Store,Pub
9,North York,0,Pizza Place,Coffee Shop,Discount Store,Butcher,Pharmacy,Concert Hall,Construction & Landscaping,Falafel Restaurant,Event Space,Ethiopian Restaurant
11,Scarborough,0,Auto Garage,Shopping Mall,Bakery,Sandwich Place,Middle Eastern Restaurant,Breakfast Spot,Ethiopian Restaurant,Event Space,Empanada Restaurant,Dive Bar


In [96]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Trail,Playground,Building,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
17,North York,1,Park,Women's Store,Diner,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
20,York,1,Convenience Store,Park,Women's Store,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
33,North York,1,Airport,Park,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Empanada Restaurant
37,Etobicoke,1,Park,Bus Line,Women's Store,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
50,North York,1,Fast Food Restaurant,Food & Drink Shop,Park,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store
51,Central Toronto,1,Dim Sum Restaurant,Park,Bus Line,Swim School,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
55,Etobicoke,1,River,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
59,Scarborough,1,Playground,Park,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
76,East York,1,Park,Pizza Place,Coffee Shop,Convenience Store,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


In [97]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,North York,2,Baseball Field,Women's Store,Filipino Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [98]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Etobicoke,3,Bank,Women's Store,Filipino Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [99]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,4,Falafel Restaurant,Coffee Shop,Caribbean Restaurant,Massage Studio,Bar,Metro Station,Women's Store,Dumpling Restaurant,Doner Restaurant,Donut Shop
10,Scarborough,4,Bar,Women's Store,Filipino Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
18,Scarborough,4,Coffee Shop,Korean Restaurant,Women's Store,Discount Store,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
